#  Differentiable Model for Muon Transport and Two-Phase Flow in Porous Media with applications to Subsurface Pollution Monitoring

#### Roland Grinis  -  Researcher at MIPT Nuclear Physics Methods lab  -  CTO at GrinisRIT (grinisrit.com)
#### Vladimir Palmin - Researcher at MIPT Nuclear Physics Methods lab, JetBrains Research
#### Danila Riazanov - Student at MIPT,  JetBrains Research trainee
#### Slava Kovalev - Student at MIPT

## 1. Introduction

Various industrial processes in geological formations, such as carbon dioxide capture sequestration:
* M. Talebian et al. *A computational model for coupled multiphysics processes of CO2 sequestration in fractured porous media.* Advances in water resources 59 (2013)

underground energy storage:
* J. Wang et al. *Overview of compressed air energy storage and technology development.* Energies 10.7 (2017)

enhanced oil recovery:
* M. Jeong et al. *Critical review on the numerical modeling of in-situ microbial enhanced oil recovery processes.* Biochemical Engineering Journal 150 (2019)

hydraulic fracturing:
* B. Chen et al. *A Review of Hydraulic Fracturing Simulation.* Archives of Computational Methods in Engineering (2021)

well disposal:
* K. P. Saripalli et al. *Modeling injection well performance during deep-well injection of liquid wastes.* Journal of Hydrology 227.1-4 (2000)


etc. could present a risk of groundwater contamination.

Acidic and non-aqueous phase liquids including chlorinated hydrocarbons, oil, supercritical $\text{CO}_2$ etc. have low solubility in brine. Their migration due to external forces must be thoroughly monitored in order to avoid long-time pollution of freshwater acquifer in the subsurface:
* M. Thangarajan et al. *Groundwater Assessment, Modeling, and Management* CRC Press, (2016)
* R. Fučík et al., *Mixed-hybrid finite element method for modelling two-phase flow in porous media.* Journal of Math-for-Industry 3, no. 2 (2011)

In this paper, we will specifically focus on geological carbon storage (GCS):
* J. Rutqvist, *The geomechanics of CO2 storage in deep sedimentary formations.* Geotechnical and Geological Engineering, 30(3) (2012)
* J.G. Wang et al., *A simple approach for the estimation of CO2 penetration depth into a caprock layer.* J. Rock Mech. Geotechn.Eng., 8 (1) (2016)

but our model applies to all of the situations mentioned above.

The use of atmospheric muons to monitor underground fluid saturation levels has been studied before, e.g.:
* F. Hivert et al., *Muography sensitivity to hydrogeological rock density perturbation: roles of the absorption and scattering on the muon flux measurement reliability.* Near Surf. Geophys., 15(2):121 (2017)
* J. Klinger et al., *Simulation of muon radiography for monitoring CO2 stored in a geological reservoir.* Int. J. Greenhouse Gas Control, 42:644 (2015)

However, the low-contrast and possibly noisy muon flux measurements require accurate and realistic modelling of the main physical processes for the inverse problem behind monitoring. Moreover, first order sensitivity information for the parameters of interest is needed to improve the analysis, see:
* C. J. Benton et al. *Optimizing geophysical muon radiography using information theory.* Geophysical Journal International 220.2 (2020)

We address those issues in present work by incorporating a differentiable programming paradigm into the implementation of the detailed physics simulations in our set-up. 

The exposition is organised in the following manner.

In section 2, we describe a model for the two-phase flow with capillary barrier effect in heterogeneuous porous media for which we rely on the mixed-hybrid ﬁnite element method (MHFEM) from:

* R. Fučík et al. *Multidimensional mixed–hybrid finite element method for compositional two-phase flow in heterogeneous porous media and its parallel implementation on GPU.* Computer Physics Communications 238 (2019)

Compared to that study, we develop a full 3D simulation and provide details for the implementation of adjoint sensitivity methods:
* L. S. Pontryagin et al., *The mathematical theory of optimal processes* (1962)

in the context of MHFEM.

In section 3, we discuss muon transport building upon the Backward Monte-Carlo (BMC) scheme from:
* V. Niess et al., *Backward Monte-Carlo applied to muon transport*, Comput. Phys. Comm.   (2018)

We re-use the spatial discretisation from MHFEM and perform sensitivity computations with respect to the media density and saturation levels following:
* R. Grinis, *Differentiable programming for particle physics simulations.* preprint arXiv:2108.10245 to appear in JETP (2021)

Finally in section 4, we put everything together to design a system for detecting  $\text{CO}_2$ leakage through the caprock layer in GCS sites. 


## 2. Adjoint sensitivity model for two-phase flow

In this section, we consider a diffusion type system of PDEs in general form relevant to modelling multi-physics multi-phase flow in heterogeneous media with barrier effects. Exact expressions for the coefficients depend on the physics and the modelling set-up. We provide a concrete example in section 4. 

### 2.1 Diffusion type evolution equations

Let $\boldsymbol{Z}(t,x, \theta) \in \mathbb{R}^{n}$ denote the primary unknown variables over $(t,x) \in \left[0,T\right] \times \varOmega \subset \mathbb{R}^{1+d}$ and control parameters $\theta \in \mathbb{R}^p$. We pose $\varOmega$ a polygonal $d$-dimensional domain (typically $d=3$) and $T$ finite simulation time.

Introduce the velocity:
$$
\boldsymbol{v}_i = - \sum_{j=1}^{n} \boldsymbol{D}_{ij} \nabla_x Z_j +  \boldsymbol{w}_i
$$
for $\boldsymbol{D}(t,x,Z,\theta) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ the diffusion tensor assumed symmetric positive-definite if non-trivial and $\boldsymbol{w}(t,x,Z,\theta) \in \mathbb{R}^n \otimes \mathbb{R}^d$ the external conservative force.

By convention we denote tensors in bold as opposed to their scalar components, e.g. $\boldsymbol{Z} = \left( Z_i \right)_{i=1}^n$, but $\boldsymbol{D} = \left( \boldsymbol{D}_{ij} \right)_{i,j=1}^n$.

The conservative flux is given by:
$$
\boldsymbol{q}_i = m_i\boldsymbol{v}_i 
$$
where $\boldsymbol{m}(t,x,Z,\theta) \in \mathbb{R}^{n}$ stands for the mobility coefficients.  

The main system governing the evolution is given by:
$$
\sum_{j=1}^{n}\left( N_{ij} \partial_t + \boldsymbol{u}_{ij} \cdot \nabla + r_{ij} \right) Z_j + \nabla_x \cdot \left( \boldsymbol{q}_i + \sum_{j=1}^{n} Z_j \boldsymbol{a}_{ij} \right) = f_i 
$$

where we the coefficients are:
* $\boldsymbol{N}(t,x,Z,\theta) \in \mathbb{R}^{n\times n}$ the damping matrix, 
* $\boldsymbol{r}(t,x,Z,\theta) \in \mathbb{R}^{n\times n}$ the reaction matrix, 
* $\boldsymbol{u}(t,x,Z,\theta) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ 
the convection tensor for the non-conservative form,
* $\boldsymbol{a}(t,x,Z,\theta) \in \mathbb{R}^{n\times n} \otimes \mathbb{R}^{d}$ 
the convection tensor for the conservative form,
* $\boldsymbol{f}(t,x,Z,\theta) \in \mathbb{R}^n$ the source/sink term

Moreover we have:
* $\boldsymbol{Z}(0,x) = \boldsymbol{Z}^{\mathcal{I}}(x,\theta)$, $\forall x \in \varOmega$ (initial conditions)
* $Z_i(t,x) = Z_i^{\mathcal{D}}(t,x,\theta)$, $\forall x \in \varGamma_{Z_i} \subset \partial\varOmega$ (Dirichlet boundary conditions)
* $\boldsymbol{q}_i \cdot \boldsymbol{n} = q_i^{\mathcal{N}}(\theta)$, $\forall x \in \varGamma_{\boldsymbol{q}_i} \subset \partial\varOmega$ with $\boldsymbol{n}$ outward unit normal to $\partial\varOmega$ (Neumann boundary conditions)

The function $\boldsymbol{Z}$ is required to be continuously differentiable in time and only continuous in space. It satisfies the evolution equations in the weak sense. We discuss the discretisation scheme to solve this system of PDEs in section 2.2.

In applications, we are typically interested to determine the parameter $\theta$ such that we match a given measurement (inverse problem) or minimise a loss function (optimisation) that we express through a functional $\mathcal{L}(\theta) = \mathcal{L}(\boldsymbol{Z}(\theta))$. To tackle this type of problems efficently, especially for higher dimensional $\theta \in \mathbb{R}^p$, we need to compute at least the first order sensitivity:
$$
\nabla_{\theta} \mathcal{L} =  \nabla_{\boldsymbol{Z}}\mathcal{L} \cdot \nabla_{\theta}\boldsymbol{Z} 
$$

However, computing the derivative $\nabla_{\theta}\boldsymbol{Z}$ is impractical for most scenarios. In section 2.3, we present an algorithm based on discrete adjoint sensitivity methods to evaluate $\nabla_{\theta} \mathcal{L}$ efficiently.

### 2.2 Mixed-hybrid finite element method

Let $\mathcal{K}_h$ be a conformal mesh for $\varOmega$ with $\mu_d(K) = \mathcal{O}(h^d)$, $\forall K \in \mathcal{K}_h$, where $\mu_d$ stands for the $d$-dimensional Lebesgue measure. Denote by $\mathcal{V}_h$ the set of vertices and by $\mathcal{F}_h$ the set of ($d-1$)-dimensional faces composing $\mathcal{K}_h$. Let us write $\mathcal{F}_K$ for the set of faces forming an element $K$.

We recall that the order zero Raviart–Thomas–Nédélec space $\textbf{RTN}_0(K)$ is spanned by piecewise linear vector fields $\omega_{K,E}$ for each $E \in \mathcal{F}_K$ whose flux is non-zero only through $E$:
$$
\omega_{K,E} \cdot \boldsymbol{n}_{K,F} = \frac{\delta_{EF}}{\mu_{d-1}(E)}
$$
and its divergence fixed to $\nabla \cdot \omega_{K,E} = \mu_d(K)^{-1}$.

We compute the spatial averages of $\boldsymbol{Z}$ for an element $K \in \mathcal{K}_h$ and a face $E \in \mathcal{F}_K$:
$$
\boldsymbol{Z}_K(t,\theta) = \frac{1}{\mu_d(K)} \int_{K}\boldsymbol{Z}(t,x,\theta)d\mu_d, \\ \boldsymbol{Z}_E(t,\theta) = \frac{1}{\mu_{d-1}(E)} \int_{E}\boldsymbol{Z}(t,x,\theta)d\mu_E
$$
respectively, with $\mu_E$ denoting the ($d-1$)-measure restricted to $E$.

Discretising time $\{ t_k \}_{k=0}^N \subset [0,T]$ with typically adaptive time-steps, we denote time-slice restrictions by $\boldsymbol{Z}^k = \boldsymbol{Z}(t_k)$. 
We should write
$\boldsymbol{Z}_{\mathcal{K}_h}^k=\{ \boldsymbol{Z}_K^k\}_{K \in \mathcal{K}_h}$ and
$\boldsymbol{Z}_{\mathcal{F}_h}^k =\{ \boldsymbol{Z}_F^k\}_{F \in \mathcal{F}_h}$
for the vectors of element and face averages respectively at time $t_k$. Those are essentially our unknowns to solve for in the discretised system.

Applying to the PDE system in section 2.1 the finite element method over $\bigotimes_{K \in \mathcal{K}_h}\textbf{RTN}_0(K)$ in spatial directions, and forward finite differences in time direction, gives us:
$$
\boldsymbol{Z}_{\mathcal{K}_h}^{k+1} = \boldsymbol{A}^k \boldsymbol{Z}_{\mathcal{F}_h}^{k+1} + \boldsymbol{R}^k
$$
The mass balance equations and barrier conditions on internal sides, together with boundary conditions on external sides, produce a linear constraint equation for the face averages:
$$
\boldsymbol{M}^k \boldsymbol{Z}_{\mathcal{F}_h}^{k+1} = \boldsymbol{b}^k
$$

We give more details about the construction of the matrices in the appendix, noting for now that they dependent on the averages $\boldsymbol{Z}_{\mathcal{K}_h}^k$, $\boldsymbol{Z}_{\mathcal{F}_h}^k$ at previous time $t_k$. For every given element $K \in \mathcal{K}_h$ only the constituent faces $F \in \mathcal{F}_K$ are involved, and only adjacent elements are considered. Therefore we deal with a sparse linear system which however leaves room for an efficient GPU implementation.

Given the properties assumed for the diffusion tensor $\boldsymbol{D}$ in previous section, martix $\boldsymbol{M}^k$ will be positive definite. The solution to the system can be obtained via various GMRES type algorithms, but a mass-lumping technique is required to stabilise the scheme.

## 3. Sensitivity analysis for muon transport 

## 4. Leakage monitoring in GCS